# Runtime Test

Before runing this test, please download models from [https://nomeroff.net.ua/models/](https://nomeroff.net.ua/models/) to **./models/** directory. 

In [1]:
import os
import sys
import json
import numpy as np
import cv2
from matplotlib import pyplot as plt
from termcolor import colored
import warnings
import time
warnings.filterwarnings('ignore')

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../')

# specify the path to Mask_RCNN if you placed it outside Nomeroff-net project
MASK_RCNN_DIR = os.path.join(NOMEROFF_NET_DIR, 'Mask_RCNN')

MASK_RCNN_LOG_DIR = os.path.join(NOMEROFF_NET_DIR, 'logs')
mode = "gpu"

sys.path.append(NOMEROFF_NET_DIR)

# Import license plate recognition tools.
from NomeroffNet import  filters
from NomeroffNet import  RectDetector
from NomeroffNet import  TextDetector
from NomeroffNet import  OptionsDetector
from NomeroffNet.DetectronDetector import  Detector
from NomeroffNet import  textPostprocessing
from NomeroffNet import  textPostprocessingAsync

rectDetector = RectDetector()

optionsDetector = OptionsDetector()
optionsDetector.load("latest")

# Initialize text detector.
textDetector = TextDetector({
    "eu_ua_2004_2015": {
        "for_regions": ["eu_ua_2015", "eu_ua_2004"],
        "model_path": "latest"
    },
    "eu_ua_1995": {
        "for_regions": ["eu_ua_1995"],
        "model_path": "latest"
    },
    "eu": {
        "for_regions": ["eu"],
        "model_path": "latest"
    },
    "ru": {
        "for_regions": ["ru", "eu-ua-fake-lnr", "eu-ua-fake-dnr"],
        "model_path": "latest" 
    },
    "kz": {
        "for_regions": ["kz"],
        "model_path": "latest"
    },
    "ge": {
        "for_regions": ["ge"],
        "model_path": "latest"
    }
},mode)

nnet = Detector()
nnet.loadModel("latest")

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8385083269637153109
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 11985617169986705529
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 48241541120
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5764041897067314697
physical_device_desc: "device: 0, name: Quadro RTX 8000, pci bus id: 0000:5e:00.0, compute capability: 7.5"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 15936487424
locality {
  bus_id: 2
  numa_node: 1
  links {
  }
}
incarnation: 4914892276377221705
physical_device_desc: "device: 1, name: Quadro RTX 5000, pci bus id: 0000:d8:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 11740275285446206480
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:1"
devic

Using TensorFlow backend.







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





Config '/var/www/centermask2/configs/numberplates/numberplate_V_39_eSE_FPN_ms_3x.yaml' has no VERSION. Assuming it to be compatible with latest v2.


In [2]:
async def test(dirName, fname):
    img_path = os.path.join(dirName, fname)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    cv_imgs_masks = nnet.detect_mask([img])
    
    for cv_img_masks in cv_imgs_masks:
        #print(np.array(cv_img_masks).shape)
        # Detect points.
        arrPoints = rectDetector.detect(cv_img_masks)

        # cut zones
        zones = await rectDetector.get_cv_zonesBGR_async(img, arrPoints, 64, 295)

        # find standart
        regionIds, stateIds, countLines = optionsDetector.predict(zones)
        regionNames = optionsDetector.getRegionLabels(regionIds)

        # find text with postprocessing by standart  
        textArr = textDetector.predict(zones, regionNames, countLines)
        textArr = await textPostprocessingAsync(textArr, regionNames)
        return textArr

In [3]:
N = 10

i = 0
j = 0
start_time = time.time()
rootDir = 'images/'
for i in np.arange(N):
    print("pass {}".format(i))
    for dirName, subdirList, fileList in os.walk(rootDir):
        for fname in fileList:
            await test(dirName, fname)
            j += 1
            #print(i, j)
    i += 1
end_time = time.time() - start_time

pass 0
[8.615511e-05 7.630590e-05 9.997868e-01 5.076263e-05]
[9.6512522e-05 9.8346907e-05 9.9974579e-01 5.9409198e-05]
[8.5000072e-05 7.3730560e-05 9.9979180e-01 4.9435723e-05]
[9.2705821e-05 7.5122873e-05 9.9978274e-01 4.9514638e-05]
[9.408523e-05 8.341422e-05 9.997726e-01 4.992809e-05]
[8.329692e-05 7.686548e-05 9.997873e-01 5.254954e-05]
[7.8264697e-05 7.3474788e-05 9.9979967e-01 4.8610069e-05]
[8.5776584e-05 7.4507290e-05 9.9979144e-01 4.8308728e-05]
[9.9909186e-01 3.5223109e-04 3.0013695e-04 2.5577529e-04]
[7.1930212e-05 7.0599126e-05 9.9981290e-01 4.4558845e-05]
[9.367228e-05 8.397177e-05 9.997682e-01 5.405997e-05]
[9.2327318e-05 7.8503406e-05 9.9977905e-01 5.0152757e-05]
[8.2412145e-05 7.3842639e-05 9.9979442e-01 4.9377144e-05]
[8.6823893e-05 7.2852810e-05 9.9979085e-01 4.9504277e-05]
[8.5521286e-05 7.8013189e-05 9.9978238e-01 5.4124797e-05]
[0.17602926 0.00420651 0.8168652  0.00289892]
[8.7232242e-05 7.3888936e-05 9.9978894e-01 4.9949289e-05]
[9.6586926e-05 7.3811585e-05 9.9978

[9.6586926e-05 7.3811585e-05 9.9978107e-01 4.8451344e-05]
[9.3572620e-05 7.9367564e-05 9.9977380e-01 5.3274296e-05]
[7.3689567e-05 6.8956899e-05 9.9980980e-01 4.7499856e-05]
[7.8022123e-05 7.2997347e-05 9.9979872e-01 5.0340244e-05]
[7.9793019e-05 7.7830824e-05 9.9978906e-01 5.3259460e-05]
[9.9960023e-01 6.8967238e-05 1.2691406e-04 2.0389857e-04]
[1.3882019e-04 1.8493398e-04 9.9959368e-01 8.2559985e-05]
[9.9956506e-01 1.4576993e-04 1.5363283e-04 1.3553977e-04]
[9.4598909e-05 7.9143305e-05 9.9977666e-01 4.9554161e-05]
[8.8955057e-05 8.3045241e-05 9.9977297e-01 5.5066459e-05]
[9.9954742e-01 1.3584783e-04 1.6257461e-04 1.5416201e-04]
[8.9680121e-05 8.0311576e-05 9.9977607e-01 5.3999123e-05]
[8.8230714e-05 8.2526130e-05 9.9977559e-01 5.3642525e-05]
[1.0321132e-04 7.8916761e-05 9.9976438e-01 5.3469092e-05]
[8.0337064e-05 7.5290111e-05 9.9979395e-01 5.0371505e-05]
pass 5
[8.615511e-05 7.630590e-05 9.997868e-01 5.076263e-05]
[9.6512522e-05 9.8346907e-05 9.9974579e-01 5.9409198e-05]
[8.5000072e

[9.6512522e-05 9.8346907e-05 9.9974579e-01 5.9409198e-05]
[8.5000072e-05 7.3730560e-05 9.9979180e-01 4.9435723e-05]
[9.2705821e-05 7.5122873e-05 9.9978274e-01 4.9514638e-05]
[9.408523e-05 8.341422e-05 9.997726e-01 4.992809e-05]
[8.329692e-05 7.686548e-05 9.997873e-01 5.254954e-05]
[7.8264697e-05 7.3474788e-05 9.9979967e-01 4.8610069e-05]
[8.5776584e-05 7.4507290e-05 9.9979144e-01 4.8308728e-05]
[9.9909186e-01 3.5223109e-04 3.0013695e-04 2.5577529e-04]
[7.1930212e-05 7.0599126e-05 9.9981290e-01 4.4558845e-05]
[9.367228e-05 8.397177e-05 9.997682e-01 5.405997e-05]
[9.2327318e-05 7.8503406e-05 9.9977905e-01 5.0152757e-05]
[8.2412145e-05 7.3842639e-05 9.9979442e-01 4.9377144e-05]
[8.6823893e-05 7.2852810e-05 9.9979085e-01 4.9504277e-05]
[8.5521286e-05 7.8013189e-05 9.9978238e-01 5.4124797e-05]
[0.17602926 0.00420651 0.8168652  0.00289892]
[8.7232242e-05 7.3888936e-05 9.9978894e-01 4.9949289e-05]
[9.6586926e-05 7.3811585e-05 9.9978107e-01 4.8451344e-05]
[9.3572620e-05 7.9367564e-05 9.9977380

### Intel(R) Core(TM) i9-9900K CPU @ 3.60GHz + Quadro RTX 8000

In [4]:
print(f"Processed {j} photos")
print(f"Time {end_time}")
print(f"One photo process {end_time/j} seconds")

Processed 260 photos
Time 46.40548133850098
One photo process 0.17848262053269606 seconds
